#rf

In [ ]:
import time
import gc
import mlflow
import optuna
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE  # Import de SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Pipeline imbalanced-learn

# Initialisation 
cv = StratifiedKFold(n_splits=5)
results_rf = []
nb_runs = 15

# Chronomètre d'entraînement
start_time = time.time()

# Définir l'expérience MLflow
mlflow.set_experiment('Random Forest Classifier avec SMOTE et Imputer dans un pipeline')

# Chargement des fichiers
X_train_eval = pd.read_csv('X_train_eval_n.csv')
y_train_eval = pd.read_csv('y_train_eval_n.csv')

# Suppression de la colonne SK_ID_CURR si présente
if 'SK_ID_CURR' in X_train_eval.columns:
    X_train_eval = X_train_eval.drop(columns=['SK_ID_CURR'])

# Conversion en float32 pour économiser la mémoire
X_train_eval = X_train_eval.astype(np.float32)

# Fonction de Random Forest avec SMOTE et Imputer pour l'optimisation Optuna
def random_forest(trial):
    # Définir les hyperparamètres avec Optuna
    n_estimators = trial.suggest_int('n_estimators', 50, 150, step=50)
    max_depth = trial.suggest_int('max_depth', 5, 10, step=1)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10, step=2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5, step=1)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])

    # Création du modèle avec pipeline (imputation, SMOTE, puis random forest)
    pipeline = ImbPipeline([
        ('imputer', SimpleImputer(strategy='mean')),  # Imputation par la moyenne
        ('smote', SMOTE(sampling_strategy='auto', random_state=42)),  # Application de SMOTE pour équilibrer les classes
        ('model', RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=False,
            n_jobs=-1,
            random_state=42
        ))
    ])

    auc_scores, acc_scores, cost_scores = [], [], []

    # Validation croisée avec SMOTE et Imputer dans le pipeline
    for train_idx, test_idx in cv.split(X_train_eval, y_train_eval):
        X_train, X_test = X_train_eval.iloc[train_idx], X_train_eval.iloc[test_idx]
        y_train, y_test = y_train_eval.iloc[train_idx].values, y_train_eval.iloc[test_idx].values

        # Entraînement du modèle avec SMOTE
        pipeline.fit(X_train, y_train)

        # Prédictions et probabilités
        y_prob = pipeline.predict_proba(X_test)[:, 1]
        y_pred = y_prob > 0.5

        # Calcul des métriques
        auc_scores.append(roc_auc_score(y_test, y_prob))
        acc_scores.append(accuracy_score(y_test, y_pred))
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        cost_scores.append(fp + 10 * fn)

    # Calcul des moyennes pour chaque métrique
    auc = np.mean(auc_scores)
    acc = np.mean(acc_scores)
    cost = np.mean(cost_scores)

    # Sauvegarde des résultats
    results_rf.append({
        "AUC": auc,
        "Accuracy": acc,
        "Business Score": cost,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf,
        "max_features": max_features
    })

    # Enregistrement dans MLflow
    with mlflow.start_run():
        mlflow.log_params(trial.params)
        mlflow.log_metrics({"AUC": auc, "Accuracy": acc, "Business Score": cost})

        # Courbe ROC
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure(figsize=(10, 7))
        plt.plot(fpr, tpr, label=f'AUC: {auc:.2f}')
        plt.title('ROC Curve')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend(loc='lower right')
        plt.savefig("roc_curve_rf.png")
        plt.close()
        mlflow.log_artifact("roc_curve_rf.png")

        mlflow.sklearn.log_model(pipeline, "random_forest_model", input_example=X_train_eval[:5])

    gc.collect()
    return cost

# Optimisation avec Optuna
study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(random_forest, n_trials=nb_runs)

# Fin du temps d'entraînement
end_time = time.time()
print(f"Temps d'entraînement total: {end_time - start_time:.2f} secondes")

# Meilleurs résultats et paramètres
best_params_rf = study_rf.best_params
best_auc_rf = max([res['AUC'] for res in results_rf])
best_acc_rf = max([res['Accuracy'] for res in results_rf])
best_cost_rf = min([res['Business Score'] for res in results_rf])

# Affichage des meilleurs paramètres
print(study_rf.best_params)
with mlflow.start_run():
    mlflow.log_params(best_params_rf)
joblib.dump(best_params_rf, 'best_params_rf.pkl')


2024/11/11 18:19:12 INFO mlflow.tracking.fluent: Experiment with name 'Random Forest Classifier avec SMOTE et Imputer dans un pipeline' does not exist. Creating a new experiment.


Temps d'entraînement total: 1648.63 secondes
{'n_estimators': 50, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'log2'}


['best_params_rf.pkl']

In [ ]:
import time
import joblib
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

######################################################################################################################################################
#2
# Entraînement final sur toutes les données avec les meilleurs paramètres
# Entraînement final sur toutes les données avec les meilleurs paramètres
start_time = time.time()

# Création du pipeline pour imputation, suréchantillonnage SMOTE, puis RandomForest
final_pipeline_rf = ImbPipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Imputation par la moyenne
    ('smote', SMOTE(random_state=42)),            # SMOTE pour équilibrer les classes
    ('model', RandomForestClassifier(**best_params_rf, n_jobs=-1, random_state=42))
])

# Entraînement du pipeline sur les données d'entraînement final
final_pipeline_rf.fit(X_final, y_final)

# Affichage du temps d'entraînement final
end_time = time.time()
print(f"Temps d'entraînement final: {end_time - start_time:.2f} secondes")

# Enregistrement du modèle final avec le pipeline
joblib.dump(final_pipeline_rf, 'random_forest_model_f.joblib')

# Sauvegarde des résultats dans un DataFrame
results_df_rf = pd.DataFrame(results_rf)

# Tri des résultats par Business Score croissant (du plus petit au plus grand)
results_df_sorted_rf = results_df_rf.sort_values(by='Business Score', ascending=True)

# Enregistrement des résultats dans un fichier CSV
results_df_sorted_rf.to_csv('results_random_forest.csv', index=False)

print("Les résultats ont été exportés dans 'results_random_forest.csv'")


c:\Users\HP\OneDrive\Documents\projet7\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Temps d'entraînement final: 130.61 secondes
Les résultats ont été exportés dans 'results_random_forest.csv'


In [10]:
######################################################################################################################################################
#3
# Recharger uniquement la colonne 'SK_ID_CURR' à partir du fichier d'origine
#X_final_pour_id = pd.read_csv(r'c:\Users\HP\OneDrive\Documents\projet7\dfs\X_final.csv', usecols=['SK_ID_CURR'])
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

# Charger les ID des clients
X_final_pour_id = pd.read_csv('X_final_n.csv', usecols=['SK_ID_CURR'])

# Aplatir y_final pour le rendre compatible avec les autres colonnes 1D
y_final = y_final.flatten()

# Prédictions finales et calcul des probabilités avec le pipeline final
y_prob_final_rf = final_pipeline_rf.predict_proba(X_final)[:, 1]
y_pred_final_rf = y_prob_final_rf > 0.5

# Calcul des métriques finales en utilisant y_final
final_auc = roc_auc_score(y_final, y_prob_final_rf)
final_accuracy = accuracy_score(y_final, y_pred_final_rf)
tn, fp, fn, tp = confusion_matrix(y_final, y_pred_final_rf).ravel()
final_cost = fp + 10 * fn

# Création du DataFrame avec les métriques pour chaque client
results_df_rf_f = pd.DataFrame({
    "SK_ID_CURR": X_final_pour_id['SK_ID_CURR'].values,
    "Probabilité": y_prob_final_rf,
    "Prédiction": y_pred_final_rf,
    "Vrai_Label": y_final
})

# Ajout des métriques globales en tant que nouvelles colonnes
results_df_rf_f["AUC"] = final_auc
results_df_rf_f["Accuracy"] = final_accuracy
results_df_rf_f["Business Score"] = final_cost

print("Métriques du modèle RandomForest final avec les meilleurs paramètres :")
print(f"AUC: {final_auc:.4f}")
print(f"Accuracy: {final_accuracy:.4f}")
print(f"Business Score: {final_cost:.4f}")

# Affichage des premières lignes du DataFrame final
results_df_rf_f.head(10)


Métriques du modèle RandomForest final avec les meilleurs paramètres :
AUC: 0.6571
Accuracy: 0.8668
Business Score: 220435.0000


,SK_ID_CURR,Probabilité,Prédiction,Vrai_Label,AUC,Accuracy,Business Score
0,202883,0.432764,False,0.0,0.657117,0.866765,220435
1,353282,0.277327,False,0.0,0.657117,0.866765,220435
2,280134,0.428717,False,0.0,0.657117,0.866765,220435
3,111527,0.353155,False,0.0,0.657117,0.866765,220435
4,362418,0.372807,False,0.0,0.657117,0.866765,220435
5,106400,0.415864,False,0.0,0.657117,0.866765,220435
6,103241,0.517210,True,0.0,0.657117,0.866765,220435
7,228257,0.572933,True,1.0,0.657117,0.866765,220435
8,179577,0.326275,False,0.0,0.657117,0.866765,220435
9,245931,0.232260,False,0.0,0.657117,0.866765,220435


In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

######################################################################################################################################################
#4
# Prédictions finales et probabilités sur X_api
y_prob_api_rf = final_pipeline_rf.predict_proba(X_api)[:, 1]
y_pred_api_rf = y_prob_api_rf > 0.5

# Calcul des métriques finales pour les données de test
api_auc = roc_auc_score(y_api, y_prob_api_rf)
api_accuracy = accuracy_score(y_api, y_pred_api_rf)
tn, fp, fn, tp = confusion_matrix(y_api, y_pred_api_rf).ravel()
api_cost = fp + 10 * fn

print("Métriques du modèle RandomForest final sur les données test :")
print(f"AUC: {api_auc:.4f}")
print(f"Accuracy: {api_accuracy:.4f}")
print(f"Business Score: {api_cost:.4f}")

# Enregistrement des métriques finales dans MLflow
with mlflow.start_run():
    mlflow.log_metrics({
        "Final AUC_t random_forest": api_auc,
        "Final Accuracy_t random_forest": api_accuracy,
        "Final Business Score_t random_forest": api_cost
    })


Métriques du modèle RandomForest final sur les données test :
AUC: 0.6762
Accuracy: 0.8706
Business Score: 2171.0000
